In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import requests
import seaborn as sbs
import dash
import plotly

In [2]:
url = "http://slnxdesa01:4998/Otrs/ObtenerListaTicket"

response = requests.get(url)

#Verificar el codifo del estado de la respuesta
if response.status_code == 200:
    data = response.json()
    
    
    reporte_otrs = pd.DataFrame(data)
    
else:
    print("Error al crear dataframe de ")
    

del response, data,url

In [3]:
#renombramos las columnas
reporte_otrs = reporte_otrs.rename(
    columns = {
        "ticket": "Ticket#",
        "titulo": "Titulo",
        "cola": "Cola",
        "estado": "Titulo",
        "cliente": "Cliente",
        "fecha": "Fecha",
        "creadoPor": "Creado Por",
        "tiempo_Trabajado": "Tiempo Trabajado",
        "downtime": "Downtime",
        "alcance": "Alcance",
        "cola_Principal": "Cola Principal",
        "cola_Secundaria": "Titulo",
        "tiempo_Trabajado_Horas": "Tiempo Trabajado (Horas)",
        "downtime_Horas": "Downtime (Horas)",
    }
)

In [4]:
#separo la columna para solamente tener la fecha sin la hora ni la region
#se dividen las columnas y obtengo la posicion 0
reporte_otrs["Fecha"] = reporte_otrs["Fecha"].str.split(' ').str[0]
reporte_otrs["Fecha"] = pd.to_datetime(reporte_otrs["Fecha"])

In [5]:
#creando downtime de cada cola principal desglosada

# Track3
track_downtime = reporte_otrs[reporte_otrs["Cola Principal"]=="Track3"].groupby("Cola",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending = False).loc[lambda x : x["Downtime"] > 0]

# BackOffice
backoffice_downtime = reporte_otrs[reporte_otrs["Cola Principal"] == "BackOffice"].groupby("Cola",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False).loc[lambda x: x["Downtime"] > 0]

# Operaciones Coto Digital
ncd_downtime = reporte_otrs[reporte_otrs["Cola Principal"] == "Operaciones Coto Digital"].groupby("Cola",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False).loc[lambda x:x["Downtime"]>0]

In [6]:
#downtime por Cola Principal
downtime_por_cola_principal = reporte_otrs.groupby("Cola Principal",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False)

#downtime por Cola
downtime_por_cola = reporte_otrs.groupby("Cola",as_index=False)["Downtime"].sum().sort_values(by="Downtime",ascending=False)

In [7]:
#cantidad de tickets por Cola Principal
conteo_tickets_por_cola_principal = reporte_otrs.groupby("Cola Principal", as_index=False)["Ticket#"].count().sort_values(by="Ticket#",ascending=False)

#cantidad de tickets por Cola
conteo_tickets_por_cola = reporte_otrs.groupby("Cola",as_index=False)["Ticket#"].count()


In [8]:
#tiempo trabajado por cola principal
tiempo_trabajado_por_cola_principal = reporte_otrs.groupby("Cola Principal",as_index=False)["Tiempo Trabajado"].sum().sort_values(by="Tiempo Trabajado",ascending=False).loc[lambda x : x["Tiempo Trabajado"]> 0]

#tiempo trabajado por cola
tiempo_trabajado_por_cola = reporte_otrs.groupby("Cola",as_index=False)["Tiempo Trabajado"].sum().sort_values(by="Tiempo Trabajado",ascending=False).loc[lambda x : x["Tiempo Trabajado"]> 0]

In [9]:
#creamos todas las agrupaciones de "Cola Principal" necesarias en una tabla
dynamics_cola_principal = reporte_otrs.groupby('Cola Principal').agg(
    Downtime_Minutos=('Downtime', 'sum'),# Suma de la columna 'Valor' como 'downtime'
    Downtime_Horas=('Downtime (Horas)','sum'),
    Tiempo_Trabajado_Minutos =  ('Tiempo Trabajado', 'sum'),
    Tiempo_Trabajado_Horas = ('Tiempo Trabajado (Horas)','sum'),
    Cantidad_Tickets=('Ticket#', 'count')   # Conteo de la columna 'Valor' como 'ticket'
).reset_index()

dynamics_cola_principal

,Cola Principal,Downtime_Minutos,Downtime_Horas,Tiempo_Trabajado_Minutos,Tiempo_Trabajado_Horas,Cantidad_Tickets
0,AIX,32,0.533,61,1.015,5
1,Administracion NT,0,0.000,8,0.133,3
2,Administracion de la operacion Coto Digital,130,2.166,913,15.216,62
3,Administracion de la operación Coto Digital,0,0.000,18,0.299,7
4,BackOffice,137,2.283,3243,54.043,165
5,Base de Datos,10,0.167,22,0.367,3
6,Comerciales,0,0.000,1225,20.416,72
7,Comunicaciones,207,3.450,237,3.950,4
8,Multistore,822,13.700,2214,36.885,101
9,Operaciones Coto Digital,810,13.499,358,5.969,34


In [12]:
#creamos todas las agrupaciones de "Cola"
dynamics_cola = reporte_otrs.groupby('Cola').agg(
    Downtime_Minutos=('Downtime', 'sum'),# Suma de la columna 'Valor' como 'downtime'
    Downtime_Horas=('Downtime (Horas)','sum'),
    Tiempo_Trabajado_Minutos =  ('Tiempo Trabajado', 'sum'),
    Tiempo_Trabajado_Horas = ('Tiempo Trabajado (Horas)','sum'),
    Cantidad_Tickets=('Ticket#', 'count')   # Conteo de la columna 'Valor' como 'ticket'
).reset_index()

dynamics_cola

,Cola,Downtime_Minutos,Downtime_Horas,Tiempo_Trabajado_Minutos,Tiempo_Trabajado_Horas,Cantidad_Tickets
0,AIX,32,0.533,29,0.482,4
1,AIX::Cambio de planificacion para optimizar el...,0,0.000,32,0.533,1
2,Administracion NT,0,0.000,8,0.133,3
3,Administracion de la operacion Coto Digital::C...,0,0.000,17,0.283,1
4,Administracion de la operacion Coto Digital::C...,0,0.000,88,1.466,7
...,...,...,...,...,...,...
102,Sitio::Promociones::Comunidad no validada corr...,0,0.000,15,0.250,1
103,Sitio::Sitio Coto Digital,0,0.000,186,3.100,7
104,Track3,35,0.584,233,3.884,9
105,Track3::APk EMC,168,2.800,122,2.033,1
